In [20]:
from pathlib import Path
import json
import pandas as pd

# app_dir = Path(__file__).parent

# EXCEL IMPORTS
registered_df = pd.read_excel("data/2025/Registered_2025.xlsx")
voted_df = "data/2025/2025.json";

# FUNCTION
def wrangle_voted():
    # Load JSON file
    with open(r"data/2025/2025.json", "r", encoding="utf-8") as f:
        data = json.load(f)

    # Flatten the nested JSON
    # data.values() returns a list of lists (one per district)
    all_records = []
    for district_list in data.values():
        all_records.extend(district_list)

    df = pd.DataFrame(all_records)
    df["votes"] = pd.to_numeric(df["votes"], errors="coerce").round()
    df['district'] = df['district'].str.strip()
    df['district'] = df['district'].replace('Mzuzu City', 'Mzimba')
    df['district'] = df['district'].replace('Blantyre City', 'Blantyre')
    df['district'] = df['district'].replace('Karonga Town', 'Karonga')
    df['district'] = df['district'].replace('Kasungu Municipality', 'Kasungu')
    df['district'] = df['district'].replace('Lilongwe Citv', 'Lilongwe')
    df['district'] = df['district'].replace('Lilongwe Rural', 'Lilongwe')
    df['district'] = df['district'].replace('Luchenza Municipality', 'Thyolo')
    df['district'] = df['district'].replace('Mangochi Municipality', 'Mangochi')
    df['district'] = df['district'].replace('Zomba City', 'Zomba')

    df['Region'] = "";

    # Mapping Regions to districts
    districts_regions = {'Mwanza': 'Southern', 'Ntcheu': 'Central', 'Neno': 'Southern', 'Nkhatabay': 'Northern', 'Chikwawa': 'Southern', 
                         'Nsanje': 'Southern', 'Nkhotakota': 'Central', 'Salima': 'Central', 'Mangochi': 'Southern', 'Machinga': 'Southern', 
                         'Dedza': 'Central', 'Karonga': 'Northern', 'Chitipa': 'Northern', 'Balaka': 'Southern', 'Zomba': 'Southern', 
                         'Mchinji': 'Central', 'Blantyre': 'Southern', 'Mzimba': 'Northern', 'Likoma': 'Northern', 'Rumphi': 'Northern', 
                         'Kasungu': 'Central', 'Mulanje': 'Southern', 'Thyolo': 'Southern', 'Chiradzulu': 'Southern', 'Lilongwe': 'Central', 
                         'Ntchisi': 'Central', 'Phalombe': 'Southern', 'Dowa': 'Central'}
    df['Region'] = df['district'].map(districts_regions)
    return df

_2025_df_final = wrangle_voted()
_2025_district_summary = pd.pivot_table(_2025_df_final, index=['district', 'Region'], values=['votes'], aggfunc='sum')

registered_df = registered_df.rename(columns={'District': 'district'})
registered_df['district'] = registered_df['district'].str.strip()
registered_df['district'] = registered_df['district'].replace('Blantyre City', 'Blantyre')
registered_df["Total"] = pd.to_numeric(registered_df["Total"], errors="coerce").round()
registered_summary = pd.pivot_table(registered_df, index=['district'], values=['Total'], aggfunc='sum')

registered_voted = pd.merge(_2025_district_summary, registered_summary, on="district", how="outer")

# _2025_df_final.to_excel("data/2025/2025.xlsx", index=False)
# registered_voted = registered_voted.sort_values(by="Total", ascending=True)
registered_voted.head(20)


,votes,Total
district,,
Balaka,134216.0,170027.0
Blantyre,383778.0,306531.0
Chikwawa,206454.0,270767.0
Chiradzulu,125023.0,146878.0
Chitipa,66958.0,0.0
Dedza,245428.0,355183.0
Dowa,247798.0,0.0
Karonga,99324.0,122430.0
Karonga Town,NaN,26702.0
